In [5]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np


# Number of locations and time periods
L = 6              # 6 fishing locations
T = 36             # 36 time periods




# p = np.array([
#     [... 36 values for River          ...],
#     [... 36 values for Cliff Top      ...],
#     [... 36 values for Mouth of River ...],
#     [... 36 values for Pond           ...],
#     [... 36 values for Sea            ...],
#     [... 36 values for Pier           ...]
# ])
p = np.array([
    [380.2352941,402.4390244,402.4390244,380.2352941,402.4390244,402.4390244,369.6296296,387.4285714,387.4285714,312.7272727,280.3636364,451.0714286,421.2698413,436.4705882,638.7755102,737.704918,1489.387755,1378.4,794.6875,1405.090909,1307.5,791.9402985,1531.47541,1433.225806,1210,1953.469388,1837.142857,538.9041096,706.1764706,906.7647059,441.8181818,548.3783784,549.7297297,388.1395349,405.1764706,405.1764706],
    [10,2151.428571,2151.428571,10,2151.428571,2151.428571,10,2553.75,2553.75,10,1678.75,1678.75,10,1775.714286,1775.714286,10,775.7142857,775.7142857,10,10,10,10,10,10,969.0909091,1775.714286,1775.714286,1292.222222,1678.75,1678.75,969.0909091,1638.823529,1638.823529,10,2151.428571,2151.428571],
    [2507.5,2507.5,2507.5,2507.5,2507.5,2507.5,1437.142857,1437.142857,1437.142857,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1033.125,1033.125,1033.125,1437.142857,1437.142857,1437.142857,4006,4006,4006,1437.142857,1437.142857,1437.142857],
    [640,936.1904762,936.1904762,640,936.1904762,936.1904762,519.1666667,754.4,706.4,401.5,520.9756098,526.5,342.7906977,387.6595745,459.5918367,572,574.2857143,706.25,600.4761905,571.372549,694.8,644.1025641,633.7777778,777.2727273,597.6470588,811.3043478,1037.391304,833,868.8888889,868.8888889,833,750.5882353,750.5882353,913.6842105,1145.263158,1145.263158],
    [986.4583333,1088.020833,1492.1875,989.0625,1090.625,1492.1875,1175.520833,1285.9375,1441.666667,994.7916667,994.7916667,1154.6875,1194.387755,1194.387755,1376.020408,1137.244898,1596.938776,1762.244898,1195.3125,1673.958333,1720.3125,1254.6875,1747.938144,1710.309278,1268.229167,1753.092784,1777.319588,1147.44898,1147.44898,1298.979592,1120.833333,1224.479167,1598.958333,986.4583333,1088.020833,1492.1875],
    [3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,3394.444444,3394.444444,3394.444444,3394.444444,2568.75,2568.75,3394.444444,2568.75,2568.75,1507.142857,1507.142857,1507.142857,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333]
])


# CONSTANTS
MAX_CASTS = 30
MIN_VISITS = 5
M = 30

# create model
model = gp.Model("Fishing")

# Variables
x = model.addVars(L, T, lb=0, vtype=GRB.CONTINUOUS, name="x")  # casts
y = model.addVars(T, vtype=GRB.BINARY, name="y")                # time period
v = model.addVars(L, vtype=GRB.BINARY, name="v")                # locations

# Constraints
# 1. Max 30 casts
model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) <= MAX_CASTS)

# 2. Exactly one time period
model.addConstr(gp.quicksum(y[t] for t in range(T)) == 1)

# 3. Link x to y
for l in range(L):
    for t in range(T):
        model.addConstr(x[l,t] <= M * y[t])

# 4. Visit at least 3 locations
model.addConstr(gp.quicksum(v[l] for l in range(L)) >= 3)

# 5. Minimum casts if visited
for l in range(L):
    model.addConstr(gp.quicksum(x[l,t] for t in range(T)) >= MIN_VISITS * v[l])

# 6. No casts at unvisited locations
for l in range(L):
    for t in range(T):
        model.addConstr(x[l,t] <= M * v[l])

# Objective: maximize expected profit
model.setObjective(gp.quicksum(p[l,t]*x[l,t] for l in range(L) for t in range(T)), GRB.MAXIMIZE)

# Solve
model.optimize()

# Print solution
def print_solution(model, x, y, v, time_names, location_names):
    if model.Status == GRB.OPTIMAL:
        print("Optimal Expected Profit:", model.ObjVal)

        # chosen time period
        chosen_time_index = max(range(T), key=lambda t: y[t].X)
        chosen_time_text = time_names[chosen_time_index]
        print("\nChosen Time Period:")
        print(f"  {chosen_time_text}")

        # visited locations
        visited_indices = [l for l in range(L) if v[l].X > 0.5]
        visited_locations = [location_names[i] for i in visited_indices]
        max_len = max(len(name) for name in visited_locations)

        print("\nVisited Locations:")
        for l in visited_indices:
            casts = x[l, chosen_time_index].X
            print(f"  {location_names[l]:<{max_len}} : {casts:>3.0f} casts")

location_names = [
    "River", "River Mouth", "Cliff Top", "Pond", "Sea", "Pier"
]


time_names = [
    "Jan 9a-4p", "Jan 4a-9a/4p-9p", "Jan 9p-4a",
    "Feb 9a-4p", "Feb 4a-9a/4p-9p", "Feb 9p-4a",
    "Mar 9a-4p", "Mar 4a-9a/4p-9p", "Mar 9p-4a",
    "Apr 9a-4p", "Apr 4a-9a/4p-9p", "Apr 9p-4a",
    "May 9a-4p", "May 4a-9a/4p-9p", "May 9p-4a",
    "Jun 9a-4p", "Jun 4a-9a/4p-9p", "Jun 9p-4a",
    "Jul 9a-4p", "Jul 4a-9a/4p-9p", "Jul 9p-4a",
    "Aug 9a-4p", "Aug 4a-9a/4p-9p", "Aug 9p-4a",
    "Sep 9a-4p", "Sep 4a-9a/4p-9p", "Sep 9p-4a",
    "Oct 9a-4p", "Oct 4a-9a/4p-9p", "Oct 9p-4a",
    "Nov 9a-4p", "Nov 4a-9a/4p-9p", "Nov 9p-4a",
    "Dec 9a-4p", "Dec 4a-9a/4p-9p", "Dec 9p-4a",
]


print_solution(model, x, y, v, time_names, location_names)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 441 rows, 258 columns and 1344 nonzeros
Model fingerprint: 0x4915a4fd
Variable types: 216 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 8801.5463900
Presolve time: 0.00s
Presolved: 441 rows, 258 columns, 1344 nonzeros
Variable types: 216 continuous, 42 integer (42 binary)

Root relaxation: objective 1.118054e+05, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 111805.417    0    3 8801.54639 111805.417  1170%     -    0s
H    0     0           